In [1]:
# Physionet
# Import dependencies
import pandas as pd
from pathlib import Path
import tensorflow as tf

In [2]:
# Preview CSV data
data = Path('../Data/fetal_health.csv')
df = pd.read_csv(data)
df.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [3]:
# Isolate feature of interest from other feature
y = df['fetal_health']
X = df.drop(columns = 'fetal_health')

In [4]:
# Split the data in training and testing data sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 48, stratify = y)

In [5]:
# Scale X data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
# Find input dimensions
input_dim = len(df.columns) - 1

In [7]:
# Define the deep learning model 
nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units = 128, activation = 'relu', input_dim = input_dim))

nn.add(tf.keras.layers.Dense(units = 64, activation = 'relu'))

nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check model summary
nn.summary()

/Users/annachen/miniconda3/envs/bootcamp/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         2,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,137 (43.50 KB)

 Trainable params: 11,137 (43.50 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Compile the Sequential model together and customize metrics
nn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Train the model
model_fit = nn.fit(X_train_scaled, y_train, epochs = 500, batch_size = 32)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose = 2)

Epoch 1/500


/Users/annachen/miniconda3/envs/bootcamp/lib/python3.11/site-packages/keras/src/losses/losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 410us/step - accuracy: 0.5418 - loss: 0.0000e+00     
Epoch 2/500
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - accuracy: 0.7873 - loss: 0.0000e+00
Epoch 3/500
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step - accuracy: 0.7624 - loss: 0.0000e+00
Epoch 4/500
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step - accuracy: 0.7782 - loss: 0.0000e+00
Epoch 5/500
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 375us/step - accuracy: 0.7847 - loss: 0.0000e+00
Epoch 6/500
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step - accuracy: 0.7700 - loss: 0.0000e+00
Epoch 7/500
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 386us/step - accuracy: 0.7638 - loss: 0.0000e+00
Epoch 8/500
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step - accuracy: 0.7854 - loss: 0.0000e+00
Epoch 9/500
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step - accuracy: 0.7839 - loss: 0.0000e+00
Epoch 10/500
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 381us/step - accuracy: 0.7715 - loss: 0.0000e+00
Epoch 11/500
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step - accuracy: 0.7801 - loss: 0.0000e+00
Epoc

/Users/annachen/miniconda3/envs/bootcamp/lib/python3.11/site-packages/keras/src/losses/losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


In [9]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

17/17 - 0s - 693us/step - accuracy: 0.7782 - loss: 0.0000e+00
Loss: 0.0, Accuracy: 0.7781955003738403
